In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlretrieve
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

# Docker

## A Very Good Walkthrough
* Part 1: [The conceptual landscape of Docker](https://towardsdatascience.com/learn-enough-docker-to-be-useful-b7ba70caeb)
* Part 2: [Useful Docker Terms](https://towardsdatascience.com/learn-enough-docker-to-be-useful-1c40ea269fa8)
* Part 3: [Dockerfile Writing](https://towardsdatascience.com/learn-enough-docker-to-be-useful-b0b44222eef5)
* Part 4: [Slim down Docker images](https://towardsdatascience.com/slimming-down-your-docker-images-275f0ca9337e)
* Part 5: [Vital Docker Commands](https://towardsdatascience.com/15-docker-commands-you-should-know-970ea5203421)
* Part 6: [Docker Volumes](https://towardsdatascience.com/pump-up-the-volumes-data-in-docker-a21950a8cd8)
* Part 7: [Docker Security](https://towardsdatascience.com/top-20-docker-security-tips-81c41dd06f57)